In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [3]:
bs,bptt = 48,50

## Language modeling

### Data

In [4]:
PATH='/home/paperspace/data/npov-edits/'

df_all = pd.read_csv(f'{PATH}all.tsv', sep='\t', names = ['Title', 'RevNo', 'NPOVTag', 'POV', 'EditorId', 'Scope', 'StrBefore', 'StrAfter', 'SentenceBefore', 'SentenceAfter'], error_bad_lines=False, warn_bad_lines = False)
#df_all = pd.read_pickle(f'{PATH}all_arxiv.pickle')

df_trn = pd.read_csv(f'{PATH}5gram-edits-train.tsv', sep='\t', names = ['Title', 'RevNo', 'NPOVTag', 'POV', 'EditorId', 'Scope', 'StrBefore', 'StrAfter', 'SentenceBefore', 'SentenceAfter'], error_bad_lines=False, warn_bad_lines = False)
df_tst = pd.read_csv(f'{PATH}5gram-edits-test.tsv', sep='\t', names = ['Title', 'RevNo', 'NPOVTag', 'POV', 'EditorId', 'Scope', 'StrBefore', 'StrAfter', 'SentenceBefore', 'SentenceAfter'], error_bad_lines=False, warn_bad_lines = False)
df_val = pd.read_csv(f'{PATH}5gram-edits-dev.tsv', sep='\t', names = ['Title', 'RevNo', 'NPOVTag', 'POV', 'EditorId', 'Scope', 'StrBefore', 'StrAfter', 'SentenceBefore', 'SentenceAfter'], error_bad_lines=False, warn_bad_lines = False)



In [56]:
df_all = df_all[df_all.POV==True]
df_trn = df_trn[df_trn.POV==True]
df_tst = df_tst[df_tst.POV==True]
df_val = df_val[df_val.POV==True]

In [65]:
df_all.txt[100:110]

14649    This has stained the word in [[German language...
15263    === Economic education ==={{NPOV}}While the Un...
15416    </ref> this isnt true==Films containing euthan...
15741    ===Reasons given against Voluntary Euthanasia:...
15879    <ref name="plato"/>* Wishes of Family: Family ...
15880    <ref name="plato"/>* Wishes of Family: Family ...
15959    * Pressure: All the arguments listed for volun...
15960    (See section [[#]]) Proponents and critics of ...
15961    ===Reasons given against Voluntary Euthanasia:...
15962    <ref>[[Hippocratic Oath]]</ref>* Moral: Some p...
Name: txt, dtype: object

In [124]:
def get_txt(df):
    return '<STR>' + df.StrBefore + ' ' + df.SentenceBefore
    #return df.SentenceBefore

#df_mb['txt'] = get_txt(df_mb)
df_all['txt'] = get_txt(df_all)
n=len(df_all); n


21348

In [121]:
"[afjas;dlkjf;la]".replace(r'[','').replace(']', '')

'afjas;dlkjf;la'

In [113]:
os.makedirs(f'{PATH}trn/yes', exist_ok=True)
os.makedirs(f'{PATH}val/yes', exist_ok=True)
os.makedirs(f'{PATH}tst/yes', exist_ok=True)
os.makedirs(f'{PATH}trn/no', exist_ok=True)
os.makedirs(f'{PATH}val/no', exist_ok=True)
os.makedirs(f'{PATH}tst/no', exist_ok=True)
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}all/tst', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)

In [114]:
df_all['txt'][:10]

213    </ref>) is a name mentioned in an inscription ...
385    <ref>[http://www.msnbc.msn.com/id/22484066/ Hu...
786    'Greater Nepal''' is the Unforgettable true hi...
787    'Greater Nepal''' is the Unforgettable true hi...
788    Since the last two hundred years the frontier ...
789    'Greater Nepal''' is the Unforgettable true hi...
790    'Greater Nepal''' is the Unforgettable true hi...
791    'Greater Nepal''' is the Unforgettable true hi...
792    'Greater Nepal''' is the Unforgettable true hi...
793    After the [[birth of Nepal]] even the passout ...
Name: txt, dtype: object

In [122]:
invalid = 0
for (i,(_,r)) in enumerate(df_all.iterrows()):
    r = random.random()
    if r < 0.1:
        dset = 'val'
    elif r < 0.2:
        dset = 'tst'
    else:
        dset = 'trn'
        
    try:
        open(f'{PATH}all/{dset}/{i}.txt', 'w').write(r['txt'])
    except:
        invalid += 1
print('%d invalid entries' % invalid)

21348 invalid entries


In [116]:
inv = 0

for (i,(_,r)) in enumerate(df_trn.iterrows()):
    try:
        open(f'{PATH}trn/yes/{i}.txt', 'w').write(r['SentenceBefore'])
    except:
        inv+=1
    try:
        open(f'{PATH}trn/no/{i}.txt', 'w').write(r['SentenceAfter'])
    except:
        inv+=1

print('%d invalid' % inv)

268 invalid


In [117]:
inv = 0

for (i,(_,r)) in enumerate(df_val.iterrows()):
    try:
        open(f'{PATH}val/yes/{i}.txt', 'w').write(r['SentenceBefore'])
    except:
        inv+=1
    try:
        open(f'{PATH}val/no/{i}.txt', 'w').write(r['SentenceAfter'])
    except:
        inv+=1

print('%d invalid' % inv)

25 invalid


In [118]:
inv = 0

for (i,(_,r)) in enumerate(df_tst.iterrows()):
    try:
        open(f'{PATH}tst/yes/{i}.txt', 'w').write(r['SentenceBefore'])
    except:
        inv+=1
    try:
        open(f'{PATH}tst/no/{i}.txt', 'w').write(r['SentenceAfter'])
    except:
        inv+=1

print('%d invalid' % inv)

29 invalid


In [50]:
from spacy.symbols import ORTH

my_tok = spacy.load('en')

my_tok.tokenizer.add_special_case('<STR>', [{ORTH: '<STR>'}])
my_tok.tokenizer.add_special_case('</STR>', [{ORTH: '</STR>'}])

def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

In [14]:
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
FILES = dict(train='trn', validation='val', test='tst')
md = LanguageModelData.from_text_files(f'{PATH}all/', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [15]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(346, 19, 1, 1557054)

In [16]:
TEXT.vocab.itos[:25]

['<unk>',
 '<pad>',
 '<eos>',
 ']',
 '[',
 'the',
 ',',
 '>',
 '<',
 'of',
 'and',
 '.',
 'str',
 'in',
 'to',
 'a',
 '-',
 '(',
 'by']

In [18]:
#print(md.trn_ds[0].text[:800])
' '.join(md.trn_ds[0].text[:150])

'<eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>'

### Train

In [5]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [58]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [59]:
learner.fit(3e-3, 1, wds=1e-6)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      4.419483   4.099431  



[4.099431]

In [60]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      3.751487   3.796194  
    1      3.583032   3.577414                            
    2      3.445612   3.519183                            
    3      3.362801   3.403717                            
    4      3.262777   3.305459                            
    5      3.16958    3.255815                            
    6      3.11106    3.244276                            



[3.2442763]

In [61]:
learner.save_encoder('npov_adam2_enc')

In [63]:
learner.fit(3e-3, 10, wds=1e-6, cycle_len=5, cycle_save_name='npov_adam3_10')

HBox(children=(IntProgress(value=0, description='Epoch', max=50), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      1.672186   2.962278  
    1      1.691198   2.948286                            
    2      1.668848   2.942816                            
    3      1.580426   2.97417                             
    4      1.554444   2.996639                            
    5      1.615516   2.939608                            
    6      1.634774   2.946914                            
    7      1.586118   2.954098                            
    8      1.521197   2.994608                            
    9      1.50279    3.012843                            
    10     1.558785   2.97014                             
    11     1.570962   2.955999                            
    12     1.510279   2.969935                            
    13     1.470292   2.985917                            
    14     1.437866   3.02467                             
    15     1.469697   2.99859                             
    16     1.470408   2

[3.1074235]

In [64]:
learner.save_encoder('npov_adam3_10_enc')

In [65]:
learner.fit(3e-3, 8, wds=1e-6, cycle_len=10, cycle_save_name='npov_adam3_5')

HBox(children=(IntProgress(value=0, description='Epoch', max=80), HTML(value='')))

epoch      trn_loss   val_loss                             
    0      1.077163   3.107204  
    1      1.177746   3.051203                            
    2      1.191216   3.072611                            
    3      1.143341   3.092905                            
    4      1.135952   3.087609                            
    5      1.091777   3.106507                            
    6      1.065367   3.102724                            
    7      1.090354   3.112051                             
    8      1.006456   3.140972                            
    9      0.988801   3.140264                             
    10     1.084744   3.10057                              
    11     1.101173   3.099823                            
    12     1.123804   3.100577                            
    13     1.093475   3.108883                            
    14     1.080248   3.131268                            
    15     1.063423   3.090158                            
    16     1.059959

[3.2595897]

In [66]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='npov_adam3_20')

HBox(children=(IntProgress(value=0, description='Epoch', max=20), HTML(value='')))

epoch      trn_loss   val_loss                             
    0      0.772339   3.266029  
    1      0.879898   3.183367                             
    2      0.871524   3.187803                             
    3      0.84951    3.189222                             
    4      0.870931   3.227053                             
    5      0.849255   3.228418                             
    6      0.839818   3.226741                             
    7      0.805588   3.242567                             
    8      0.763004   3.255096                             
    9      0.765558   3.263549                             
    10     0.717341   3.267666                             
    11     0.72184    3.261291                             
    12     0.685168   3.279845                             
    13     0.693035   3.292675                             
    14     0.652053   3.295908                             
    15     0.647651   3.30353                              
    16 

[3.296725]

In [67]:
learner.save_encoder('npov_adam3_20_enc')

In [68]:
learner.save('npov_adam3_20')

### Test

In [69]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [70]:
m=learner.model

In [71]:
s="""<STR>claim</STR>People claim"""

In [72]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [73]:
sample_model(m,"Many believe that")

...' ' it '' is often used to be a [ [ gay ] ] author who has written three of the three [ [ soviet union ] ] as well as the [ [ state department ] ] 's [ [ office of the united states ] ] . 

In [74]:
sample_model(m,"<CAT> cscv <SUMM> algorithms that")

..., on 2005 june 2010 , the group were named as [ [ mary ] ] , [ [ st ( uk ] ] , [ [ county for the israeli conflict ] ] . <eos> 

In [75]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> on ")

...the peace of a section of our country : the ' ' south '' israeli forces are not built . <eos> 

In [225]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> on ")

...the performance of wireless networks <eos> 

In [226]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> towards ")

...a new approach to image classification <eos> 

In [227]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> towards ")

...a new approach to the analysis of wireless networks <eos> 

### Sentiment

In [9]:
SENT_PATH='/home/paperspace/data/npov-edits/'
EMB_PATH='/home/paperspace/fastai/data/wikitext-103/models/'

In [10]:
TEXT = pickle.load(open(f'{EMB_PATH}/TEXT.pkl','rb'))

In [11]:
class NpovDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['yes', 'no']:
            for fname in glob(os.path.join(path, label, '*.txt')):
                with open(fname, 'r') as f: text = f.readline()
                examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [12]:
ARX_LABEL = data.Field(sequential=False)
splits = NpovDataset.splits(TEXT, ARX_LABEL, SENT_PATH, train='trn', test='val')

In [18]:
md2 = TextData.from_splits(SENT_PATH, splits, bs)

In [19]:
#            dropout=0.3, dropouti=0.4, wdrop=0.3, dropoute=0.05, dropouth=0.2)

In [20]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_at_6(preds,targs):
    precision, recall, _ = precision_recall_curve(targs==2, preds[:,2])
    print(recall[precision>=0.6][0])
    return recall[precision>=0.6][0]

In [25]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.65, wdrop=0.5, dropoute=0.1, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.clip=25.

In [26]:
m3.load_encoder(f'adam3_10_enc.h5')
lrs=np.array([1e-4,1e-3,1e-2])

In [27]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

 18%|█▊        | 134/741 [00:01<00:08, 68.70it/s, loss=1.16]


RuntimeError: given sequence has an invalid size of dimension 2: 0

 18%|█▊        | 134/741 [00:20<01:35,  6.38it/s, loss=1.16]

In [87]:
m3.fit(lrs, 2, metrics=[accuracy], cycle_len=4, cycle_save_name='npov')

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

 44%|████▎     | 242/556 [00:08<00:11, 26.97it/s, loss=1.11]

Exception in thread Thread-222:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



epoch      trn_loss   val_loss   accuracy                   
    0      1.119212   1.090271   0.34375   
    1      1.108808   1.045843   0.507473                   
    2      1.101405   1.060506   0.492867                   
    3      1.103208   1.082475   0.451766                   
    4      1.102478   1.057407   0.476902                   
    5      1.100153   1.060554   0.462976                   
    6      1.099601   1.070916   0.446671                   
    7      1.101185   1.068756   0.45856                    



[1.0687557, 0.4585597826086957]

In [92]:
prec_at_6(*m3.predict_with_targs())

0.1561439239646979


0.1561439239646979

In [90]:
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=2, cycle_save_name='npov')

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

 38%|███▊      | 213/556 [00:08<00:14, 23.81it/s, loss=1.1]

Exception in thread Thread-232:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



epoch      trn_loss   val_loss   accuracy                   
    0      1.099763   1.062939   0.533628  
    1      1.10222    1.069235   0.501698                   
    2      1.109209   1.07631    0.389606                   
    3      1.096849   1.085217   0.44837                    
    4      1.113687   1.071457   0.496264                   
    5      1.112742   1.073944   0.477242                   
    6      1.113939   1.087442   0.451087                   
    7      1.12685    1.068301   0.477921                   



[1.0683014, 0.4779211956521739]

In [91]:
prec_at_6(*m3.predict_with_targs())

0.1561439239646979


0.1561439239646979